In [1]:
from subprocess import call
import time
import glob
import os
import sys
import tifffile
from skimage import io
import numpy as np
import nibabel as nib
import multiprocessing as mp
from scipy import signal
import gc
import time

In [2]:
def is_file_empty(file_path):
    """ Check if file is empty by confirming if its size is 0 bytes"""
    # Check if file exist and it is empty
    return os.path.exists(file_path) and os.stat(file_path).st_size == 0
def Register_single_image(Mov_name,template_name):        
    output_name = Mov_name.replace('.tif','_LongReg')      
    if not os.path.exists(output_name+'.nii') and is_file_empty(output_name+'.nii'):
        #job_string = "antsRegistration -d 3 --float 1 -o [OutImg, OutImg.nii] -n WelchWindowedSinc -w [0.005,0.995] -u 1 -r [FixImg,MovImg, 1] -t rigid[0.1] -m MI[FixImg,MovImg,1,32, Regular,0.25] -c [1000x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x1x1x0vox -t Affine[0.1] -m MI[FixImg,MovImg,1,32, Regular,0.25] -c [1000x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x1x1x0vox -t SyN[0.05,6,0.5] -m CC[FixImg,MovImg,1,2] -c [100x500x200x50,1e-7,5] -f 8x4x2x1 -s 2x2x1x0vox -v 0"            
        job_string = "antsRegistration -d 3 --float 1 -o [OutImg,OutImg.nii] -n WelchWindowedSinc -w [0.05,0.95]  -u 0 - r [FixImg,MovImg] -t Rigid[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t Affine[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t SyN[0.1] -m CC[FixImg,MovImg,1,2] -c 100x50x20x10 -f 8x4x2x1 -s 3x2x1x0 -v 1"        
        job_string = job_string.replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
        call([job_string],shell=True)  
def Register_single_image_forced(Mov_name,template_name):        
    output_name = Mov_name.replace('.tif','_LongReg') 
    job_string = "antsRegistration -d 3 --float 1 -o [OutImg,OutImg.nii] -n WelchWindowedSinc -w [0.05,0.95]  -u 0 - r [FixImg,MovImg] -t Rigid[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t Affine[0.1] -m MI[FixImg,MovImg,1,32,Regular,0.25] -c 500x200x100x50 -f 8x4x2x1 -s 3x2x1x0 -t SyN[0.1] -m CC[FixImg,MovImg,1,2] -c 100x50x20x10 -f 8x4x2x1 -s 3x2x1x0 -v 1"        
    job_string = job_string.replace('OutImg',output_name).replace('FixImg',template_name).replace('MovImg',Mov_name)
    call([job_string],shell=True)  

In [4]:
def MakeListAndHyperstack(tif_file_folder):
    tif_list=glob.glob(tif_file_folder+'/*.tif')
    tif_list.sort()
    if not (os.path.exists(tif_file_folder+'/3Dreg/template.tif') and not is_file_empty(tif_file_folder+'/3Dreg/'+tif_file_folder.split('/')[-2]+'_0.tif')):
        temp=[]
        for idx_nb,file in enumerate(tif_list):
            if idx_nb==0:
                temp=io.imread(file,plugin='pil')
            else:
                temp=np.concatenate((temp,io.imread(file,plugin='pil')),axis=0)
            
        file_name=tif_file_folder.split('/')[-1]
        range2=int(file_name.split('range')[-1].split('_')[0])
        step=int(file_name.split('step')[-1].split('_')[0])
        TrueSlices=(range2/step)+1;
        dims=temp.shape
        temp=np.reshape(temp,[dims[0]/TrueSlices,TrueSlices,dims[1],dims[2]])

        #Computing the difference between points from frame to frame should give the point with least changes
        #std_movie=temp[:,0:200].std(axis=(2,3))
        number_of_frames_to_check=50
        std_movie=np.diff(temp[0:number_of_frames_to_check].reshape([number_of_frames_to_check,temp.shape[1],temp.shape[2]*temp.shape[3]]),axis=-1)
        idx_template=np.argmin(std_movie,axis=0)
        #ignores the 0s and compute the mean index of the lowest changes
        idx_template=int(idx_template[idx_template>0].ravel().mean())

        #tmp_idx=np.argmin(signal.detrend(std_movie))
        #mean image of the 3 points surrounding the minimum movement
        template=temp[idx_template-1:idx_template+2].mean(axis=0)
        directory = os.getcwd()
        try:
            new_dir=tif_file_folder+'/3Dreg'
            os.mkdir(new_dir)
        except:
            print('directory exists')
            
        os.chdir(new_dir)
        template_name=new_dir+'/template.tif'
        tifffile.imsave(template_name,template.astype(np.uint16))
        img_seq_list=list()
         
        for img_nb in range(0,temp.shape[0]):
            img_name=new_dir+'/'+tif_file_folder.split('/')[-1]+'_'+str(img_nb)+'.tif'
            tifffile.imsave(img_name,temp[img_nb].astype(np.uint16))
            img_seq_list.append(img_name)
        del temp,std_movie
        gc.collect()
    return(tif_list)

In [16]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    print(tif_file_folder)
    img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
    template_name=tif_file_folder+'/3Dreg/template.tif'
    p=mp.Pool(processes = N)
    result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
    for ind_result in result:
        ind_result.get()    

/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish1_ENSFed_5DPF_range160_step5_exposure15_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish1_ENSGF_5DPF_range140_step5_exposure17_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish1_ENS_5DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish2_ENSFed_5DPF_range120_step5_exposure20_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish2_ENSGF_5DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish2_ENS_5DPF_range130_step5_exposure18_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish3_ENSFed_5DPF_range100_step5_exposure23_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish3_ENSGF_5DPF_range100_step5_exposure23_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish3_ENS_5DPF_range140_step5_exposure17_power60/
/mnt

In [17]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    print(tif_file_folder)
    img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
    template_name=tif_file_folder+'/3Dreg/template.tif'
    p=mp.Pool(processes = N)
    result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
    for ind_result in result:
        ind_result.get()    

/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish1_ENSFed_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish1_ENSGF_6DPF_range150_step5_exposure16_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish1_ENS_6DPF_range100_step5_exposure23_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish2_ENSFed_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish2_ENSGF_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish2_ENS_6DPF_range140_step5_exposure17_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish3_ENSFed_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish3_ENSGF_6DPF_range140_step5_exposure17_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730/GV_20200730_fish3_ENS_6DPF_range110_step5_exposure21_power60/
/mnt

In [19]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    print(tif_file_folder)
    img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
    template_name=tif_file_folder+'/3Dreg/template.tif'
    p=mp.Pool(processes = N)
    result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
    for ind_result in result:
        ind_result.get()    

/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish1_ENSFed_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish1_ENSGF_6DPF_range150_step5_exposure16_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish1_ENS_6DPF_range100_step5_exposure23_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish2_ENSFed_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish2_ENSGF_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish2_ENS_6DPF_range140_step5_exposure17_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish3_ENSFed_6DPF_range110_step5_exposure21_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish3_ENSGF_6DPF_range140_step5_exposure17_power60/
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731/GV_20200730_fish3_ENS_6DPF_range110_step5_exposure21_power60/
/mnt

In [9]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish2_ENSGF_3DPF_range100_step5_exposure25_power60_1/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish3_ENSGF_3DPF_range100_step5_exposure25_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish3_ENS_4DPF_range140_step5_exposure17_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish5_ENS_3DPF_range100_step5_exposure24_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish6_ENSGF_3DPF_range100_step5_exposure24_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish6_ENS_3DPF_range140_step5_exposure17_power60/ failed


In [10]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish3_ENSGF_4DPF_range100_step5_exposure23_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish3_ENS_4DPF_range140_step5_exposure17_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish4_ENSGF_4DPF_weirdmount_range100_step5_exposure23_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish5_ENSGF_4DPF_range100_step5_exposure23_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish5_ENS_4DPF_range100_step5_exposure23_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish8_ENSGF_4DPF_range120_step5_exposure20_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200728/GV_20200727_fish8_ENSGF_4DPF_range140_step5_exposure17_power60/ failed


In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish1_ENSGF_5DPF_range140_step5_exposure17_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish1_ENS_5DPF_range110_step5_exposure21_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish2_ENSGF_5DPF_range110_step5_exposure21_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish3_ENSFed_5DPF_range100_step5_exposure23_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish3_ENSGF_5DPF_range100_step5_exposure23_power60/ failed
/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200729/GV_20200729_fish4_ENSGF_5DPF_range100_step5_exposure23_power60/ failed


In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200730'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [ ]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200731'
os.chdir(os.path.dirname(Fish_files_folder))
N = int((mp.cpu_count()/4)-2)
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
for i,tif_file_folder in enumerate(Fish_list):    
    try:
        tif_list=MakeListAndHyperstack(tif_file_folder)
        img_seq_list=glob.glob(tif_file_folder+'/3Dreg/*.tif')
        template_name=tif_file_folder+'/3Dreg/template.tif'
        p=mp.Pool(processes = N)
        result=[p.apply_async(Register_single_image, (img_name,template_name)) for img_name in img_seq_list]
        for ind_result in result:
            ind_result.get()
    except:
        print(tif_file_folder + ' failed')

In [5]:
Fish_files_folder='/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727'
os.chdir(os.path.dirname(Fish_files_folder))
Fish_list=glob.glob(Fish_files_folder+'/*/')
Fish_list.sort()
tif_file_folder=Fish_list[0]
MC_img_list=glob.glob(tif_file_folder+'/3Dreg/*.nii')
len(MC_img_list)

1200

In [60]:
sizes=np.zeros(len(MC_img_list))
for i,img_name in enumerate(MC_img_list):
    sizes[i]=os.stat(img_name).st_size
sum(sizes<sizes.max())

0

In [53]:
sizes.max()

34560352.0

In [56]:
np.where(sizes<34560352)[0].astype('int')

array([445])

In [58]:
temp=[MC_img_list[j].replace('_LongReg.nii','.tif') for j in np.where(sizes<34560352)[0].astype(int)]
temp

['/mnt/f/ENSDEV20-Q3066/Data/GF_Fed2020/20200727/GV_20200727_fish1_ENSGF_3DPF_range120_step5_exposure20_power60//3Dreg/GV_20200727_fish1_ENSGF_3DPF_range120_step5_exposure20_power60_984.tif']

In [ ]:
temp

In [45]:
template_name=tif_file_folder+'/3Dreg/template.tif'
N = int((mp.cpu_count()/4)-2)
p=mp.Pool(processes = N)
result=[p.apply(Register_single_image_forced, (img_name,template_name)) for img_name in temp]
for ind_result in result:
    ind_result.get()

AttributeError: 'NoneType' object has no attribute 'get'

In [59]:
for img_name in temp:
    Register_single_image_forced(img_name,template_name)

In [61]:
name=MC_img_list[0]    
base_img=nib.load(name)
base_img=np.squeeze(np.asarray(base_img.get_fdata(),dtype='float32'))
base_img.shape

(640, 540, 25)

In [30]:
start=time.time()
time.sleep(10)
time.time()-start

10.010842084884644

In [38]:
os.path.exists(C1_name)

False

In [1]:
frames=np.zeros((len(MC_img_list)*base_img.shape[2],base_img.shape[1],base_img.shape[0]), dtype='uint16')
for img_nb,C1_name in enumerate(MC_img_list):
    print(img_nb)
    img_nb=int(C1_name.split('power60_')[1].split('_Long')[0])
    img_temp=[]
    start=time.time()
    while not img_temp:
        try:
            img_temp=nib.load(C1_name)
        except nib.filebasedimages.ImageFileError:
            if time.time() - start > 3600:
                print("Time out while accessing :" + os.path.basename(C1_name))
                pass
            else:
                time.sleep(30)
        except:
            print("Unexpected error: " + os.path.basename(C1_name))
            raise
    img_temp=img_temp.get_fdata()    
    img_temp=np.squeeze(np.asarray(img_temp,dtype='uint16'))
    frames[base_img.shape[2]*(img_nb):base_img.shape[2]+base_img.shape[2]*(img_nb),:,:]=np.transpose(img_temp,(2,1,0))
tifffile.imsave(name.replace('.nii','_3D.tif'),frames)

NameError: name 'np' is not defined

In [8]:
frames.shape

(30000, 540, 640)

In [42]:
tif_file_folder='D:/Gilles_HS_ENS/3Dreg'
list_img = glob.glob(tif_file_folder+'/*Reg.nii')    
list_img.sort()
name=list_img[0]    
base_img=nib.load(name)
base_img=np.squeeze(np.asarray(base_img.get_fdata(),dtype='float32'))

IndexError: list index out of range

In [12]:
frames=np.zeros((len(list_img)*21,base_img.shape[1],base_img.shape[0]), dtype='uint16')
for img_nb,C1_name in enumerate(list_img):    
    img_temp=nib.load(C1_name)
    img_temp=img_temp.get_fdata()    
    img_temp=np.squeeze(np.asarray(img_temp,dtype='uint16'))
    frames[21*(img_nb):21+21*(img_nb),:,:]=np.transpose(img_temp,(2,1,0))
tifffile.imsave(name.replace('.nii','_3D.tif'),frames)

In [43]:
tif_file_folder='D:/Gilles_HS_ENS/3Dreg'
list_img = glob.glob(tif_file_folder+'/*.tif')    
list_img.sort()
list_img

['D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_0.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_10.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_100.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1000.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1001.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1002.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1003.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1004.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_1005.tif',
 'D:/Gilles_HS_ENS/3Dreg\\GV_

In [45]:
name=list_img[0]    
base_img=tifffile.imread(name)
base_img=np.squeeze(np.asarray(base_img,dtype='uint16'))

In [46]:
name

'D:/Gilles_HS_ENS/3Dreg\\GV_20200727_fish2_ENS_3DPF_range120_step5_exposure20_power60_0.tif'

In [49]:
frames=np.zeros((len(list_img)*25,base_img.shape[1],base_img.shape[-1]), dtype='uint16')
for img_nb,C1_name in enumerate(list_img):    
    img_temp=tifffile.imread(C1_name)         
    frames[25*(img_nb):25+25*(img_nb),:,:]=img_temp
tifffile.imsave(name.replace('_0.tif','_3D.tif'),frames)

In [35]:
img_temp.shape

(21, 540, 640)

In [37]:
frames.shape

(25200, 540, 21)

In [38]:
base_img.shape

(21, 540, 640)